In this code I want to implemnet simple transformer model to read shakespeare and spit out some text.

In [2]:
# let us first get the test
# Let's open a webpage, download the text file and read it
import urllib.request
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = urllib.request.urlopen(url)
data_in = response.read().decode('utf8')

# print first 1000 characters of the raw text
# print(data_in[:100])

In [ ]:
# simple character level tokenizer
# First, let us create a list of all the unique characters in the text and sort them
chars = sorted(list(set(data_in)))
#print(chars)

# the vocab size is the number of unique characters
vocab_size = len(chars)

# create a mapping to map each character to a unique integer
chtoi = {c: i for i, c in enumerate(chars)}
itoch = {i: c for i, c in enumerate(chars)}

# create an encode function to encode the text into a list of integers ( took help from Andrej's video)
def encode(text):
    return [chtoi[c] for c in text]

# create a decode function to decode the list of integers into text
def decode(text):
    return ''.join([itoch[i] for i in text])

# okay, now let us encode the text into a list of integers
data_en = encode(data_in)


In [ ]:
# Okay, time to start training
# first, put the data into a torch tensor
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import numpy as np



In [ ]:
# create a tensor of the encoded text
data = torch.tensor(data_en, dtype=torch.long)

In [ ]:
# create self attention class
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        
        assert (self.head_dim * heads == embed_size), "Embed size needs to be divisible by heads"
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias = False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias = False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias = False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
        
    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        
        # split embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)
        
        values = self.values(values) # (N, value_len, heads, head_dim)
        keys = self.keys(keys) # (N, key_len, heads, heads_dim)
        queries = self.queries(queries) # (N, query_len, heads, heads_dim)
        
        # energy = QK^T
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        
        # queries shape: (N, query_len, heads, heads_dim)
        # keys transpose shape: (N, heads, heads_dim, key_len)
        # energy: (N, heads, query_len, key_len)
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        
        # attention = softmax(energy / (d ** (1/2)))
        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3)
        
        # out = attention * V
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N   
            , query_len 
            , self.heads * self.head_dim
        )

        # concatenate heads together and put through final linear layer
        out = self.fc_out(out)

        return out
    
# create a transformer 